<a href="https://colab.research.google.com/github/CarlosChavezErazo/Netflix_bd/blob/main/delfosti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CARGA Y EXPLORACION DE DATOS

In [ ]:
# Cargamos las librerias a usar
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
# Cargamos la BD desde GitHub
df = pd.read_csv('https://raw.githubusercontent.com/CarlosChavezErazo/Netflix_bd/refs/heads/main/netflix_titles.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


# LIMPIEZA DE DATOS

In [ ]:
# Canviamos a formato fecha y reestructuramos la duracion

df['date_added'] = pd.to_datetime(df['date_added'],format='mixed')
df[['duration_value','duration_unit']] = df['duration'].str.extract(r'(\d+)\s*(\w+)');
df['duration_value'] = pd.to_numeric(df['duration_value'])

print(df[['duration_value', 'duration_unit']])

      duration_value duration_unit
0               90.0           min
1                2.0       Seasons
2                1.0        Season
3                1.0        Season
4                2.0       Seasons
...              ...           ...
8802           158.0           min
8803             2.0       Seasons
8804            88.0           min
8805            88.0           min
8806           111.0           min

[8807 rows x 2 columns]


In [ ]:
# Limpieza de las columnas
df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce')
df['title'] = df['title'].str.title()
df['country'] = df['country'].str.strip().str.title()
df['rating'] = df['rating'].str.strip().str.upper()

In [ ]:
# Eliminar duplicados
print("Duplicate rows before:", df.duplicated().sum())
df = df.drop_duplicates()
if 'description' in df.columns:
    df.drop(columns=['description'], inplace=True)

# Verificar directior y reparto
df['dir_cast'] = df['director'] + '---' + df['cast'] #combinamos
counts = df['dir_cast'].value_counts() # cuenta
filtered_counts = counts[counts >= 3] # filtra mas de 3 veces
filtered_values = filtered_counts.index # Extrae la columna combinada

# Agregamos indice a la nueva columna combinada
lst_dir_cast = list(filtered_values)

#Agregamos "null" alos espacion vacios
dict_direcast = dict()
for i in lst_dir_cast:
    director, cast = i.split('---')
    dict_direcast[director] = cast

for i in range(len(dict_direcast)):
    director_name = list(dict_direcast.items())[i][0]
    cast_name = list(dict_direcast.items())[i][1]

    df.loc[(df['director'].isna()) & (df['cast'] == cast_name), 'director'] = director_name

df.loc[df['director'].isna(),'director'] = 'null'

Duplicate rows before: 0


In [ ]:
# Agregamos "null" a las columnas "cast" y "country"
directors = df['director']
countries = df['country']

pairs = zip(directors, countries)
director_country = dict(pairs)

for director, country in director_country.items():
    df.loc[(df['country'].isna()) & (df['director'] == director), 'country'] = country

df.loc[df['country'].isna(), 'country'] = 'null'
df.loc[df['cast'].isna(), 'cast'] = 'null'

In [ ]:
# Eliminamos de las  filas vacías de "date_added, rating, duration"
df.drop(df[df['date_added'].isna()].index, axis=0, inplace=True)
df.drop(df[df['rating'].isna()].index, axis=0, inplace=True)
df.drop(df[df['duration'].isna()].index, axis=0, inplace=True)

In [ ]:
# Verificamos "date_added" anterior al año de lanzamiento
invalid_dates_count = sum(df['date_added'].dt.year < df['release_year'])
print("Registros con fechas inconsistentes:", invalid_dates_count)
df.loc[(df['date_added'].dt.year < df['release_year']), ['date_added', 'release_year']]

# Verificamos que no haya inconsistencias
inconsistent_years = df['date_added'].dt.year < df['release_year']
df.loc[inconsistent_years, 'release_year'] = df.loc[inconsistent_years, 'date_added'].dt.year
df.iloc[[1551, 1696, 2920, 3168]]
print('Ocurrencia',sum(df['date_added'].dt.year < df['release_year']))


Records with inconsistent dates: 14
Inconsistent records preview:
Occurence 0


In [ ]:
# Nueva columnas del año en el que se añadio a Netflix y la antiguedad
df['year_added'] = pd.to_datetime(df['date_added'], errors='coerce').dt.year
df['year_added']
df['age_at_added'] = df['year_added'] - df['release_year']
df['age_at_added']

# Agregamos la columna is_new_release
df['is_new_release'] = (df['age_at_added'] <= 1).astype(int)
df['is_new_release'] = df['is_new_release'].map({1: 'Nuevo lanzamiento', 0: 'Antiguo lanzamiento'})
df['is_new_release']



,release_decade
0,2020
1,2020
2,2020
3,2020
4,2020
...,...
8802,2000
8803,2010
8804,2000
8805,2000


# VALIDAMOS LOS DATOS Y GUARDAMOS

In [ ]:
# Eliminamos columnas temporales añadidos anteriormente, verificando las inconsistencias de los datos
if 'dir_cast' in df.columns:
    df.drop(columns=['dir_cast'], inplace=True)

print("Tipos de datos:")
print(df.dtypes[['date_added', 'duration_value']])

invalid_years = df[df['date_added'] < pd.Timestamp('1997-01-01')]
print(f"Registros con años añadidos antes del1997: {len(invalid_years)}")
print(invalid_years[['title', 'release_year', 'date_added']].head())

missing_summary = df[['title', 'director', 'cast', 'country', 'date_added', 'release_year', 'duration_value']].isna().sum()
print("Valores faltantes:")
print(missing_summary)

print("Ejemplo de datos limpios y validados:")
print(df.sample(5))

df = df.reset_index(drop=True)

Data types check:
date_added        datetime64[ns]
duration_value           float64
dtype: object
Records with added year before 1997: 0
Empty DataFrame
Columns: [title, release_year, date_added]
Index: []
Missing values check (important fields):
title             0
director          0
cast              0
country           0
date_added        0
release_year      0
duration_value    0
dtype: int64
Sample of cleaned and validated dataset:
     show_id     type                                              title  \
7862   s7863  TV Show                                            Revenge   
2943   s2944    Movie                          The Ballad Of Lefty Brown   
7778   s7779    Movie  Power Rangers Super Megaforce: The Legendary B...   
3509   s3510    Movie                                        Kabir Singh   
8181   s8182    Movie                                 The Adventure Club   

                              director  \
7862                              null   
2943              

In [ ]:
from google.colab import files
df.to_csv('netflix_titles_clean.csv', index=False)
files.download('netflix_titles_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>